# Chained Wood function with Broyden banded constraints


$$\left\{  
\begin{array}{lll} 
\min_x F(x) \\ 
\text{s.c.}\\
 c_k(x)=0 \quad \text{ pour } 1\leq k \leq n-7
\end{array} \right.$$

$$F(x)=\sum_{i=1}^{n/2} \left[ 100(x_{2i-1}^2-x_{2i})^2 + (x_{2i-1}-1)^2+  90(x_{2i+1}^2-x_{2i+2})^2 + (x_{2i+1}-1)^2 + 10(x_{2i}+x_{2i+2}-2)^2 + (x_{2i}-x_{2i+2})^2/10\right]$$

$$c_k(x) = (2+x_{k+5}^2)x_{k+5}+1 + \sum_{i=k-5}^{k+1}x_i(1+x_i)$$

In [ ]:
using BenchmarkTools, ForwardDiff, Distributions, JuMP, Ipopt

In [ ]:
include("../src/enlsip_functions_castor2.jl")

In [ ]:
n = 20 # 1000 20, needs to be >= 8
m = 6 * (div(n,2)-1)
nb_eq = n-7
nb_constraints = nb_eq

In [ ]:
# Residuals

function r(x::Vector)
    n = length(x)
    N = div(n,2) - 1
    s = √(10)
    
    rx1 = [10(x[2i-1]^2 - x[2i]) for i=1:N]
    rx2 = [x[2i-1] - 1 for i=1:N]
    rx3 = [3s*(x[2i+1]^2 - x[2i+2]) for i=1:N]
    rx4 = [x[2i+1]-1 for i=1:N]
    rx5 = [s*(x[2i] + x[2i+2] - 2) for i=1:N]
    rx6 = [(x[2i] - x[2i+2])*(1/s) for i=1:N]
    
    return [rx1;rx2;rx3;rx4;rx5;rx6]
end

resCW = ResidualsEval(0)

function (resCW::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resCW.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resCW.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
function c(x::Vector)
    n = length(x)
    cx = [(2+5x[k+5]^2)*x[k+5] + 1 + sum(x[i]*(1+x[i]) for i=max(k-5,1):k+1) for k=1:n-7]
    return cx
end
    
consCW = ConstraintsEval(0)

function (consCW::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consCW.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consCW.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [ ]:
include("../src/enlsip_functions_castor2.jl")

x0 = [(mod(i,2) == 1 ? -2. : 1.) for i=1:n]
# x0 = -.5*ones(n)
# x0 = rand(n)
enlsipCW = enlsip(x0,resCW,consCW,n,m,nb_eq,nb_constraints)
x_julia = enlsipCW.sol

## Résolution avec Ipopt

In [ ]:
x0 = [(mod(i,2) == 1 ? -2.0 : 1.0) for i=1:n] #* 1.05
N = div(n,2) - 1

model = Model(Ipopt.Optimizer)
@variable(model, x[i=1:n], start = x0[i])

for k=1:n-7
    @NLconstraint(model, (2+5x[k+5]^2)*x[k+5] + 1 + sum(x[i]*(1+x[i]) for i=max(k-5,1):k+1) == 0)
end

@NLobjective(model, Min, sum(100(x[2i-1]^2 - x[2i])^2 + (x[2i-1]-1)^2 + 90*(x[2i+1]^2-x[2i+2])^2 + 
        (x[2i+1]-1)^2 + 10*(x[2i]+x[2i+1]-2)^2 + (1/10)*(x[2i] - x[2i+2])^2 for i=1:N))

JuMP.optimize!(model)
x_ipopt = value.(model[:x])

In [ ]:
# Solution Enlsip-Julia avec solution ipopt en point de départ

enlsipCW = enlsip(x_ipopt,resCW,consCW,n,m,nb_eq,nb_constraints)
x_julia_doped = enlsip.CW.sol 

## Solution ENLSIP-Fortran

In [ ]:
x_fortran = vec([ -0.32400972154965557 -0.11241914579121411 -0.36084158184805698 0.11241481217194124 -8.5535381647864075E-002 -0.28358212095924523 -0.20403792796606549 -0.24913846278393031 -0.22151483408926775 -0.14021517868460462 -6.5210223599222211E-002 -8.0575250433167739E-002 -4.5317994168056493E-002 -9.8225879658166773E-002 -7.5902345104595466E-003 -9.6575547086696287E-003 -8.7910872447834701E-002 -0.12198573228729126 -1.4162735124366688 2.0153088741929333 ]);

## Visualisation

In [ ]:
x0 = [(mod(i,2) == 1 ? -2. : 1.) for i=1:n]
enlsipCW = enlsip(x0,resCW,consCW,n,m,nb_eq,nb_constraints)

str_to_array = (str::String, T::DataType=Float64) -> parse.(T, split(chop(str; head=1, tail=1), ','))

df = DataFrame(CSV.File("iterates.csv", delim=";"));

In [ ]:
# Calcul du point le plus éloigné de la droite entre le point de départ et la solution


x_0 = str_to_array(first(df[!,:x]))
x_N = str_to_array(last(df[!,:x]))
xNmx0 = x_N - x_0
nrm_xNmx0 = norm(xNmx0)

dist = 0
x_loin = x_0
x_loin_proj = x_0

# println("||xN-x0|| = $nrm_xNmx0")

for str_x ∈ df[!,:x]
    x = str_to_array(str_x)
#     println("\n\ncoeff :",(dot(x-x_0,xNmx0) / nrm_xNmx0)," pour x = $x")
    x_proj = (dot(x-x_0,xNmx0) / dot(xNmx0,xNmx0)) * xNmx0 + x_0
#     println(x_proj)
#     println(dot(x-x_proj,xNmx0))
    dist_proj = norm(x-x_proj)
    if dist_proj > dist
        dist = dist_proj
        x_loin = x
        x_loin_proj = x_proj
    end
end

In [ ]:
using Plots; plotlyjs()

In [ ]:

# Square sum of residuals

function ssr(x::Vector)
    rx = r(x)
    return dot(rx,rx)
end

function ssc(x::Vector)
    cx = c(x) 
    return min(dot(cx,cx),2e4)
end

# Rays of the cone {λd + μd̄ | (λ,μ) ∈ [0,ρ]²}

d = x_N - x_0 # x_loin - (3x_0-x_N)/2
d̄ = x_loin - x_loin_proj
ρ = 2*nrm_xNmx0 #  norm(xNmx0*3/2)

λ_inf = -.5 #-.5 
λ_sup = 1.5 #1.5
μ_inf = -1. #-.5
μ_sup = 1. #1.5

scal_d = range(λ_inf,λ_sup,500)
scal_d̄ = range(μ_inf,μ_sup,500)

C =  [x_0 + λ*d + μ*d̄ for μ ∈ scal_d̄, λ ∈ scal_d]

z = ((ssr).(C))
cz = (ssc).(C)
# @show extrema(z)


# Projection des itérés
iterates = str_to_array.(df[!,:x])
λ_iterates = [dot(x-x_0,xNmx0) * (1/dot(xNmx0,xNmx0)) for x ∈ iterates]
μ_iterates = [dot(x-x_0,d̄) * (1/dot(d̄,d̄)) for x ∈ iterates]
residuals_iterates = ssr.(iterates) 
N = length(iterates)

# Plot de la surface

# scatter3d!([1],[0],[enlsipOsborne2.obj_value],markershape=:diamond)

scatter3d(λ_iterates, μ_iterates, residuals_iterates, mz=range(1,N),mc=:blues,colorbar=:none)
plot!(λ_iterates, μ_iterates, residuals_iterates, mz=range(1,N),lw=3,lc=:green,colorbar=:none)
surface!(scal_d,scal_d̄, z, xlabel="λ", ylabel="μ", zlabel="Résidus", fz = cz, fc = :acton,
    xlims=(λ_inf,λ_sup), ylims=(μ_inf,μ_sup),zlims=(0,2e4), colorbar=:best)

In [ ]:
# Square sum of residuals

function ssr(x::Vector)
    rx = r(x)
#     return min(10,dot(rx,rx))
    return dot(rx,rx)
end

res_line = t -> ssr(t*x_julia+(1-t)*x_ipopt)

t = range(0,1,100)

plot(t,res_line.(t))

In [ ]:
# Square sum of residuals

function ssr(x::Vector)
    rx = r(x)
#     return min(10,dot(rx,rx))
    return dot(rx,rx)
end

function ssc(x::Vector)
    cx = c(x) 
    return dot(cx,cx)
end

# Rays of the cone {λd + μd̄ | (λ,μ) ∈ [0,ρ]²}

d = x_julia_ipopt - x_ipopt 
d̄ = x_julia - x_ipopt
ρ = 2*norm(x_julia - x_ipopt) #  norm(xNmx0*3/2)

λ_inf = -1. #-.5 
λ_sup = 2. #1.5
μ_inf = -1. #-.5
μ_sup = 2. #1.5

scal_d = range(λ_inf,λ_sup,100)
scal_d̄ = range(μ_inf,μ_sup,100)

C =  [x_ipopt + λ*d + μ*d̄ for μ ∈ scal_d̄, λ ∈ scal_d]

z = ((ssr).(C))
cz = (ssc).(C)
# cz = (ssc).(C)
# @show extrema(z)


# Projection des itérés
# iterates = str_to_array.(df[!,:x])
# λ_iterates = [dot(x-x_0,xNmx0) * (1/dot(xNmx0,xNmx0)) for x ∈ iterates]
# μ_iterates = [dot(x-x_0,d̄) * (1/dot(d̄,d̄)) for x ∈ iterates]
# residuals_iterates = ssr.(iterates) 
# N = length(iterates)

# Plot de la surface

# scatter3d!([1],[0],[enlsipOsborne2.obj_value],markershape=:diamond)

surface(scal_d,scal_d̄, cz,zlims=(0,0.1))

In [ ]:
# Square sum of residuals

function ssr(x::Vector)
    rx = r(x)
#     return min(10,dot(rx,rx))
    return dot(rx,rx)
end

function ssc(x::Vector)
    cx = c(x) 
    return dot(cx,cx)
end

# Rays of the cone {λd + μd̄ | (λ,μ) ∈ [0,ρ]²}

d = x_julia_ipopt - x_ipopt 
d̄ = x_julia - x_ipopt
ρ = 2*norm(x_julia - x_ipopt) #  norm(xNmx0*3/2)

λ_inf = -1. #-.5 
λ_sup = 2. #1.5
μ_inf = -1. #-.5
μ_sup = 2. #1.5

scal_d = range(λ_inf,λ_sup,100)
scal_d̄ = range(μ_inf,μ_sup,100)

C =  [x_ipopt + λ*d + μ*d̄ for μ ∈ scal_d̄, λ ∈ scal_d]

z = ((ssr).(C))
cz = (ssc).(C)
# cz = (ssc).(C)
# @show extrema(z)


# Projection des itérés
# iterates = str_to_array.(df[!,:x])
# λ_iterates = [dot(x-x_0,xNmx0) * (1/dot(xNmx0,xNmx0)) for x ∈ iterates]
# μ_iterates = [dot(x-x_0,d̄) * (1/dot(d̄,d̄)) for x ∈ iterates]
# residuals_iterates = ssr.(iterates) 
# N = length(iterates)

# Plot de la surface

# scatter3d!([1],[0],[enlsipOsborne2.obj_value],markershape=:diamond)

surface(scal_d,scal_d̄, z,zlims = (400,1000))